In [ ]:
## Imports
import pandas as pd
import json
from sklearn.model_selection import train_test_split

In [ ]:
file_path_all = "../climate_classifier/scored_data/votes_data_all.pkl"
file_path_climate = "../climate_classifier/scored_data/votes_data_climate.pkl"

df = pd.read_pickle("../climate_classifier/scored_data/votes_data_all.pkl")

### Train-Test Split
Making the Train-Test split of our files to avoid data leakage and comparability between our models with the seed random_state=42

In [ ]:
c_train, c_test = train_test_split(df[df['y_pred']=='climate'], test_size=.20, shuffle=True, random_state=42)
c_train.reset_index(drop=True, inplace=True)
c_test.reset_index(drop=True, inplace=True)

In [ ]:
nc_train, nc_test = train_test_split(df[df['y_pred']=='non-climate'], test_size=.20, shuffle=True, random_state=42)

In [ ]:
train = nc_train.append(c_train)
test = nc_test.append(c_test)

In [ ]:
train = train.sample(frac=1)
test = test.sample(frac=1)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
## Climate only splits
#c_train.to_pickle("c_train.pkl")
#c_test.to_pickle("c_test.pkl")

In [ ]:
## All data splits
#train.to_pickle("train.pkl")
#test.to_pickle("test.pkl")

## Preparing data for training by party and politician

In [ ]:
df = df.drop_duplicates(subset='description_text').reset_index(drop=True)
df = df[1:-1]

### Expanded view by party

In [ ]:
view = df[['date','description_text','votes', 'vote_caller']]

In [ ]:
expanded_view = pd.DataFrame(columns=['date', 'description', 'politician', 'vote'])


for index, row in view.iterrows():
    
    
    try:
        temp = pd.DataFrame(view['votes'][index])
        temp['description'] = view['description_text'][index]
        temp['date'] = view['date'][index]
        
        
        expanded_view = temp.append(expanded_view)
    
    except:
        temp['description'] = view['description_text'][index]
        temp['date'] = view['date'][index]
        print("error")
    
expanded_view

In [ ]:
expanded_view.to_pickle("../climate_classifier/scored_data/votes_data_all_expanded_politician.pkl")

### Expanded View by Party

In [ ]:
view = df[['date', 'description_text', 'votes_by_party', 'vote_caller']]

In [13]:
expanded_view = pd.DataFrame(columns=['date', 'description', 'party', 'vote'])


for index, row in view.iterrows():
    
    
    try:
        temp = pd.DataFrame(view['votes_by_party'][index])
        temp['description'] = view['description_text'][index]
        temp['date'] = view['date'][index]
        
        expanded_view = temp.append(expanded_view)
    
    except:
        temp['description'] = view['description_text'][index]
        temp['date'] = view['date'][index]
        print("error")
        
expanded_view

In [ ]:
expanded_view.drop(columns=['party', 'vote'], inplace=True)

In [ ]:
## Save to csv
import json

lst = []

for i in expanded_view.parties.unique():
    lst.append(i.split(" (")[0])

with open('party_lst.json', 'w') as outfile:
    json.dump(lst, outfile) 

dir = "by_party/"
for i in expanded_view.parties.unique():
    name = i.split(" (")[0]
    df = expanded_view[expanded_view.parties == i]
    #df.to_pickle(dir+name+".pkl", protocol=4)
    df.to_csv(dir+name+".csv")